In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar


--2022-01-24 21:02:24--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  5.48MB/s    in 0.2s    

2022-01-24 21:02:25 (5.48 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)

In [5]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating","helpful_votes","total_votes", "vine","verified_purchase"])

In [6]:
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3O9SGZBVQBV76|          5|            0|          0|   N|                Y|
| RKH8BNC3L5DLF|          5|            0|          0|   N|                Y|
|R2HLE8WKZSU3NL|          2|            1|          1|   N|                Y|
|R31U3UH5AZ42LL|          5|            0|          0|   N|                Y|
|R2SV659OUJ945Y|          4|            0|          0|   N|                Y|
| RA51CP8TR5A2L|          5|            0|          0|   N|                Y|
| RB2Q7DLDN6TH6|          5|            1|          1|   N|                Y|
|R2RHFJV0UYBK3Y|          1|            1|          5|   N|                N|
|R2Z6JOQ94LFHEP|          5|            1|          2|   N|                Y|
| RX27XIIWY5JPB|          4|            0|          0|   N|     

In [7]:
from pyspark.sql.functions import col
votes_df= vine_df.filter(vine_df.total_votes >= 20)


In [8]:
votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R14W2VCHHK5V7W|          5|           19|         20|   N|                Y|
|R1S3T57O3OYT5S|          5|           19|         20|   N|                Y|
|R1BTWIBLYYVOV7|          5|           30|         30|   N|                Y|
| R6F9VY91ADPLA|          1|            8|         30|   N|                N|
|R3PXNV89DFIXKV|          5|           35|         37|   N|                Y|
|R2ZF9NYVT3J7D6|          5|           19|         22|   N|                Y|
|R20NYA5V0UF9NE|          5|           27|         28|   N|                Y|
|R2X8FZRUOS8R8C|          4|           25|         26|   N|                Y|
|R2D8IMBVX3XCLF|          1|           14|         20|   N|                Y|
|R25UD9TA63L3Q8|          5|           25|         27|   N|     

In [9]:
filtered_votes_df = votes_df.filter((vine_df.helpful_votes/vine_df.total_votes) >= .50)
filtered_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R14W2VCHHK5V7W|          5|           19|         20|   N|                Y|
|R1S3T57O3OYT5S|          5|           19|         20|   N|                Y|
|R1BTWIBLYYVOV7|          5|           30|         30|   N|                Y|
|R3PXNV89DFIXKV|          5|           35|         37|   N|                Y|
|R2ZF9NYVT3J7D6|          5|           19|         22|   N|                Y|
|R20NYA5V0UF9NE|          5|           27|         28|   N|                Y|
|R2X8FZRUOS8R8C|          4|           25|         26|   N|                Y|
|R2D8IMBVX3XCLF|          1|           14|         20|   N|                Y|
|R25UD9TA63L3Q8|          5|           25|         27|   N|                Y|
|R2RB1HML8N712P|          5|           26|         28|   N|     

In [11]:
#Filering paid reviews
filtered_paid_reviews = filtered_votes_df.filter(filtered_votes_df.vine == "Y")

In [13]:
#Filering unpaid reviews
filtered_unpaid_reviews = filtered_votes_df.filter(filtered_votes_df.vine == "N")

In [19]:
 # Total count of stars
 from pyspark.sql.functions import sum,avg,max,min,mean,count
stars_count_total = filtered_votes_df.groupby("star_rating").agg(count("star_rating")).withColumnRenamed("count(star_rating)", "star_count")

stars_count_total.show()

+-----------+----------+
|star_rating|star_count|
+-----------+----------+
|          1|      1400|
|          3|       771|
|          5|      4347|
|          4|      1352|
|          2|       539|
+-----------+----------+



In [20]:
# Paid analysis

stars_count_paid = filtered_paid_reviews.groupby("star_rating").agg(count("star_rating")).withColumnRenamed("count(star_rating)", "star_count")
stars_count_paid = stars_count_paid.withColumnRenamed("star_count", "star_count_paid")
stars_count_paid.show()

+-----------+---------------+
|star_rating|star_count_paid|
+-----------+---------------+
|          1|              2|
|          3|              7|
|          5|             15|
|          4|             20|
|          2|              3|
+-----------+---------------+



In [21]:
# UnPaid analysis
stars_count_unpaid = filtered_unpaid_reviews.groupby("star_rating").agg(count("star_rating")).withColumnRenamed("count(star_rating)", "star_count")
stars_count_unpaid.show()

+-----------+----------+
|star_rating|star_count|
+-----------+----------+
|          1|      1398|
|          3|       764|
|          5|      4332|
|          4|      1332|
|          2|       536|
+-----------+----------+



In [37]:
import pyspark.sql.functions as F  
#Join the 2 tables and include total
joined_df=stars_count_paid.join(stars_count_unpaid,on="star_rating", how="inner")
joined_df_sum = joined_df.withColumn('total', joined_df.star_count + joined_df.star_count_paid)
joined_df_percentage = joined_df_sum.withColumn('% of paid', joined_df_sum.star_count_paid / joined_df_sum.agg(F.sum("star_count_paid")).collect()[0][0]*100) 
joined_df_percentage = joined_df_percentage.withColumn('% of unpaid', joined_df_sum.star_count / joined_df_percentage.agg(F.sum("star_count")).collect()[0][0]*100) 
joined_df_percentage.show()

+-----------+---------------+----------+-----+------------------+------------------+
|star_rating|star_count_paid|star_count|total|         % of paid|       % of unpaid|
+-----------+---------------+----------+-----+------------------+------------------+
|          1|              2|      1398| 1400|  4.25531914893617| 16.71848839990433|
|          3|              7|       764|  771|14.893617021276595|   9.1365701985171|
|          5|             15|      4332| 4347|31.914893617021278| 51.80578808897393|
|          4|             20|      1332| 1352|  42.5531914893617|15.929203539823009|
|          2|              3|       536|  539| 6.382978723404255| 6.409949772781632|
+-----------+---------------+----------+-----+------------------+------------------+



In [ ]:
#Calcular el total de reviews
Total_reviews = joined_df_percentage.agg(sum(joined_df_percentage.star_count_paid+joined_df_percentage.star_count))
Total_reviews.show()

+-----------------------------------+
|sum((star_count_paid + star_count))|
+-----------------------------------+
|                               8409|
+-----------------------------------+

